# Here we merge our cleaned data

## Importing libraries

In [1]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date

In [2]:
## Loading the Datasets
df_industries = pd.read_csv('Prepared Frames/industry_data.csv')
df_companies = pd.read_csv('Prepared Frames/company_data.csv')
df_balance = pd.read_csv('Prepared Frames/balance_data.csv')
df_cashflow = pd.read_csv('Prepared Frames/cashflow_data.csv')